In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
from IPython.display import clear_output
import json
from statistics import mode

In [2]:
# Array de UMA lista de listas

In [3]:
def MAV_EMG(df, WINDOW_SIZE):
    MAV_values = []
    for j in range(8):
        MAV_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i][:WINDOW_SIZE]['EMG_s'+ str(j)].reset_index(drop=True)
                MAV_temp.append(mean(abs(samples_temp)))
        MAV_values.append(MAV_temp)
    return MAV_values

def WL_EMG(df, WINDOW_SIZE):
    WL_values = []
    for j in range(8):
        WL_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i][:WINDOW_SIZE-1]['EMG_s'+ str(j)].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i][1:WINDOW_SIZE]['EMG_s'+ str(j)].reset_index(drop=True)
                WL_temp.append(sum(abs(samples_temp_plus-samples_temp)))
        WL_values.append(WL_temp)
    return WL_values

def SSC_EMG(df, WINDOW_SIZE, limiar):
    SSC_values = []
    for j in range(8):
        SSC_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][2:WINDOW_SIZE].reset_index(drop=True)
                samples_temp_minus = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-2].reset_index(drop=True)
                SSC_temp.append(sum(((samples_temp - samples_temp_minus)*(samples_temp - samples_temp_plus)>limiar).astype(int)))
        SSC_values.append(SSC_temp)
    return SSC_values

def ZC_EMG(df, WINDOW_SIZE, limiar):
    ZC_values = []
    for j in range(8):
        ZC_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE].reset_index(drop=True)
                op1 = (samples_temp*samples_temp_plus) >= limiar
                op2 = (samples_temp - samples_temp_plus) >= limiar
                ZC_temp.append(sum(op1!=op2))
        ZC_values.append(ZC_temp)
    return ZC_values

#--------------------------------------------------------------------------------------------------------------------------

def IEMG_EMG(df, WINDOW_SIZE):
    IEMG_values = []
    for j in range(8):
        IEMG_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                IEMG_temp.append(sum(abs(samples_temp)))
        IEMG_values.append(IEMG_temp)
    return IEMG_values

def DASDV_EMG(df, WINDOW_SIZE):
    DASDV_values = []
    for j in range(8):
        DASDV_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE].reset_index(drop=True)
                DASDV_temp.append(sum(pow(samples_temp_plus-samples_temp,2))/len(samples_temp))
                if (np.isnan(sum(pow(samples_temp_plus-samples_temp,2))/len(samples_temp))):
                    print(len(samples_temp))
                    print(len(samples_temp_plus))
                    print(len(df[df['signal'] == i]))
                    print()
        DASDV_values.append(DASDV_temp)
    return DASDV_values

def WAMP_EMG(df, WINDOW_SIZE, limiar):
    WAMP_values = []
    for j in range(8):
        WAMP_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) > WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE-1].reset_index(drop=True)
                samples_temp_plus = df[df['signal'] == i]['EMG_s'+ str(j)][1:WINDOW_SIZE].reset_index(drop=True)
                WAMP_temp.append(sum(abs(samples_temp-samples_temp_plus)>=limiar))
        WAMP_values.append(WAMP_temp)
    return WAMP_values

def AR4_EMG(df, WINDOW_SIZE):
    AR4_values = []
    for j in range(8):
        AR4_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                rho, sigma = sm.regression.linear_model.burg(samples_temp, order=4)
                AR4_temp.append(rho.tolist())
        AR4_values.append(AR4_temp)
    return AR4_values

def VORDER_EMG(df, WINDOW_SIZE, v):
    VORDER_values = []
    for j in range(8):
        VORDER_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) >= WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                VORDER_temp.append(abs(pow(sum(pow(samples_temp,v))/len(samples_temp),1/v)))
        VORDER_values.append(VORDER_temp)
    return VORDER_values

#--------------------------------------------------------------------------------------------------------------------------

def VAR_EMG(df, WINDOW_SIZE):
    VAR_values = []
    for j in range(8):
        VAR_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) > WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                VAR_temp.append(sum(pow(samples_temp,2))/(len(samples_temp)-1))
        VAR_values.append(VAR_temp)
    return VAR_values

def RMS_EMG(df, WINDOW_SIZE):
    RMS_values = []
    for j in range(8):
        RMS_temp = []
        for i in range(1,max(df['signal']) + 1):
            if(len(df[df['signal'] == i]) > WINDOW_SIZE):
                samples_temp = df[df['signal'] == i]['EMG_s'+ str(j)][:WINDOW_SIZE].reset_index(drop=True)
                RMS_temp.append(pow(sum(pow(samples_temp,2))/len(samples_temp),1/2))
        RMS_values.append(RMS_temp)
    return RMS_values

In [4]:
def GROUP_1 (df, WINDOW_SIZE, limiar, label):
    # MAV, WL, SSC E ZC
    MAV_LIST = MAV_EMG(df, WINDOW_SIZE)
    WL_LIST  = WL_EMG(df, WINDOW_SIZE)
    SSC_LIST = SSC_EMG(df, WINDOW_SIZE,limiar)
    ZC_LIST  = ZC_EMG(df, WINDOW_SIZE,limiar)
    dict_list = []
    for j in range(min([max(df['signal']), 10])):
        if(len(df[df['signal'] == j+1]) >= WINDOW_SIZE):
            entry = [label]
            sensor_list = []
            for i in range(8):
                sensor_list.append(MAV_LIST[i][j])
                sensor_list.append(WL_LIST[i][j])
                sensor_list.append(SSC_LIST[i][j])
                sensor_list.append(ZC_LIST[i][j])
                if(np.isnan(MAV_LIST[i][j])):
                    print('MAV')
                if(np.isnan(WL_LIST[i][j])):
                    print('WL')
                if(np.isnan(SSC_LIST[i][j])):
                    print('SSC')
                if(np.isnan(ZC_LIST[i][j])):
                    print('ZC')
            entry.append(sensor_list)
            dict_list.append(entry)
    df_car = pd.DataFrame(dict_list)
    df_car.columns = ['label', 'feature']
    return df_car

In [5]:
def GROUP_2 (df, WINDOW_SIZE, limiar, v, label):
#    IEMG_LIST   = IEMG_EMG(df, WINDOW_SIZE)
    DASDV_LIST  = DASDV_EMG(df, WINDOW_SIZE)
#    WAMP_LIST   = WAMP_EMG(df, WINDOW_SIZE, limiar)
#    VORDER_LIST = VORDER_EMG(df, WINDOW_SIZE, v)
    AR4_LIST = AR4_EMG(df, WINDOW_SIZE)
    dict_list = []
    for j in range(min([max(df['signal']), 10])):
        if(len(df[df['signal'] == j+1]) >= WINDOW_SIZE):
            entry = [label]
            sensor_list = []
            for i in range(8):
                sensor_list.append(IEMG_LIST[i][j])
                sensor_list.append(DASDV_LIST[i][j])
                sensor_list.append(WAMP_LIST[i][j])
                sensor_list.append(VORDER_LIST[i][j])
                sensor_list.extend(AR4_LIST[i][j])
                if(np.isnan(IEMG_LIST[i][j])):
                    print('IEMG_LIST')
                if(np.isnan(DASDV_LIST[i][j])):
                    print('DASDV_LIST')
                if(np.isnan(WAMP_LIST[i][j])):
                    print('WAMP_LIST')
                if(np.isnan(VORDER_LIST[i][j])):
                    print('VORDER_LIST')
                if(any(np.isnan(AR4_LIST[i][j]))):
                    print('AR4_LIST')
            entry.append(sensor_list)
            dict_list.append(entry)
    df_car = pd.DataFrame(dict_list)
    df_car.columns = ['label', 'feature']
    return df_car

In [2]:
def file_segmented_path(subject_number, gesture_number, file_number):
#   return "C:/Users/User/Desktop/nao_excluir/Matheus_Taborda/Matheus/Subjects_segmented/s_"+str(subject_number)+"/G"+str(gesture_number) +"/Emg_"+str(file_number)+".csv"
    return "D:/matc/Documents/Projects/Subjects_segmented/s_"+str(subject_number)+"/G"+str(gesture_number) +"/Emg_"+str(file_number)+".csv"

In [3]:
group_number = 1
subject_number = 1

WINDOW_SIZE = 150 # 300 crasha

# subject_number = 0

best_parameters = {}

df_car = pd.DataFrame(columns=['label', 'feature'])
for subject_number in range(5):
    for gesture_number in range(1,6):
        for file_number in range(10):
            
            file_segmented  = file_segmented_path(subject_number, gesture_number, file_number)

            df = pd.read_csv(file_segmented)
            df.drop('Unnamed: 0', axis = 1, inplace = True)
            label  ='G' + str(gesture_number)
            # Normalizando valores:
            for i in range(8):
                df['EMG_s'+str(i)] = df['EMG_s'+str(i)].apply(lambda x: x/128)
            if (max(df.signal)>0):
                if group_number == 1:
                    df_temp = GROUP_1(df, WINDOW_SIZE, 0.001, label)
                if group_number == 2:
                    df_temp = GROUP_2(df, WINDOW_SIZE, 0.001, 3, label)
                df_car = df_car.append(df_temp.copy())

NameError: name 'pd' is not defined

In [54]:
best_parameters.update({'tree_group_number'     : group_number,
                        'tree_WINDOW_SIZE'      : WINDOW_SIZE,
                        'precision_tree'        : 0,
                        'bag_group_number'      : group_number,
                        'bag_WINDOW_SIZE'       : WINDOW_SIZE,
                        'precision_bag'         : 0,
                        'rf_group_number'       : group_number,
                        'rf_WINDOW_SIZE'        : WINDOW_SIZE,
                        'precision_rf'          : 0,
                        'svm_group_number'      : group_number,
                        'svm_WINDOW_SIZE'       : WINDOW_SIZE,
                        'precision_svm'         : 0,
                        'lda_group_number'      : group_number,
                        'lda_WINDOW_SIZE'       : WINDOW_SIZE,
                        'precision_lda'         : 0})

In [7]:
def calculate_general_precision(cm):
    precision = 0
    correct = 0
    total = sum(sum(cm))
    for i in range(len(cm)):
        correct = sum([cm[i][i], correct])
    precision = correct/total
    return precision 

In [56]:
tree_depth = 8

bag_n_estimators = 30
bag_max_samples = 50

rf_depth = 7

svm_gamma = 0.001
svm_C = 100

In [57]:
df_train = df_car

In [58]:
X_train, X_test, y_train, y_test = train_test_split(
df_train['feature'].to_list(), df_train['label'].to_list(), test_size=0.1, random_state=42)

In [59]:
for tree_depth in range(2,15,2):
    tree_clf = DecisionTreeClassifier(max_depth=tree_depth)
    tree_clf.fit(X_train, y_train)
    cm_tree = confusion_matrix(tree_clf.predict(X_test), y_test)
    precision_tree = calculate_general_precision(cm_tree)
    if precision_tree > best_parameters['precision_tree']:
        best_parameters.update({'tree_depth'     : tree_depth ,
                                'precision_tree' : precision_tree})

In [60]:
for bag_n_estimators in range(10,140,20):
    for bag_max_samples in range(25,151,25):
        bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = bag_n_estimators, max_samples = bag_max_samples, bootstrap = True, n_jobs=-1)
        bag_clf.fit(X_train, y_train)
        cm_bag = confusion_matrix(bag_clf.predict(X_test), y_test)
        precision_bag  = calculate_general_precision(cm_bag)
        if precision_bag > best_parameters['precision_bag']:
            best_parameters.update({'bag_n_estimators': bag_n_estimators ,
                                    'bag_max_samples' : bag_max_samples ,
                                    'precision_bag'   : precision_bag})

In [61]:
for rf_depth in range(2,15,2):
    rf_clf = RandomForestClassifier(max_depth=rf_depth, random_state=42)
    rf_clf.fit(X_train, y_train)
    cm_rf = confusion_matrix(rf_clf.predict(X_test), y_test)
    precision_rf   = calculate_general_precision(cm_rf)
    if precision_rf > best_parameters['precision_rf']:
        best_parameters.update({'rf_depth'     : rf_depth ,
                                'precision_rf' : precision_rf})

In [62]:
for svm_gamma in [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:
    for svm_C in [1, 10, 100, 1000, 10000, 100000, 10000000]:
        svm_clf = SVC(kernel="rbf", gamma= svm_gamma, C=svm_C)
        svm_clf.fit(X_train, y_train)
        cm_svm = confusion_matrix(svm_clf.predict(X_test), y_test)
        precision_svm  = calculate_general_precision(cm_svm)
        if precision_svm > best_parameters['precision_svm']:
            best_parameters.update({'svm_gamma'       : svm_gamma ,
                                    'svm_C'           : svm_C ,
                                    'precision_svm'   : precision_svm})

In [63]:
best_parameters

{'tree_group_number': 1,
 'tree_WINDOW_SIZE': 150,
 'precision_tree': 0.7647058823529411,
 'bag_group_number': 1,
 'bag_WINDOW_SIZE': 150,
 'precision_bag': 0.7352941176470589,
 'rf_group_number': 1,
 'rf_WINDOW_SIZE': 150,
 'precision_rf': 0.8739495798319328,
 'svm_group_number': 1,
 'svm_WINDOW_SIZE': 150,
 'precision_svm': 0.8823529411764706,
 'lda_group_number': 1,
 'lda_WINDOW_SIZE': 150,
 'precision_lda': 0,
 'tree_depth': 10,
 'bag_n_estimators': 110,
 'bag_max_samples': 150,
 'rf_depth': 12,
 'svm_gamma': 0.0001,
 'svm_C': 100}

In [64]:
count = 1

for group_number in [1, 2]:
    for WINDOW_SIZE in range(50,251,50):
        df_car = pd.DataFrame(columns=['label', 'feature'])
        for subject_number in range(5):
            for gesture_number in range(1,6):
                for file_number in range(10):
                    file_segmented  = file_segmented_path(subject_number, gesture_number, file_number)

                    df = pd.read_csv(file_segmented)
                    df.drop('Unnamed: 0', axis = 1, inplace = True)
                    label  ='G' + str(gesture_number)
                    # Normalizando valores:
                    for i in range(8):
                        df['EMG_s'+str(i)] = df['EMG_s'+str(i)].apply(lambda x: x/128)
                    if (max(df.signal)>0):
                        if group_number == 1:
                            df_temp = GROUP_1(df, WINDOW_SIZE, 0.001, label)
                        if group_number == 2:
                            df_temp = GROUP_2(df, WINDOW_SIZE, 0.001, 3, label)
                        df_car = df_car.append(df_temp.copy())
                    
        df_train = df_car
        
        X_train, X_test, y_train, y_test = train_test_split(
        df_train['feature'].to_list(), df_train['label'].to_list(), test_size=0.1, random_state=42)

        tree_clf = DecisionTreeClassifier(max_depth=best_parameters['tree_depth'])
        tree_clf.fit(X_train, y_train)
        cm_tree = confusion_matrix(tree_clf.predict(X_test), y_test)
        precision_tree = calculate_general_precision(cm_tree)
        if precision_tree > best_parameters['precision_tree']:
            best_parameters.update({'tree_group_number'     : group_number,
                                    'tree_WINDOW_SIZE'      : WINDOW_SIZE,
                                    'precision_tree' : precision_tree})

        bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = best_parameters['bag_n_estimators'], max_samples = best_parameters['bag_max_samples'], bootstrap = True, n_jobs=-1)
        bag_clf.fit(X_train, y_train)
        cm_bag = confusion_matrix(bag_clf.predict(X_test), y_test)
        precision_bag  = calculate_general_precision(cm_bag)
        if precision_bag > best_parameters['precision_bag']:
            best_parameters.update({'bag_group_number': group_number,
                                    'bag_WINDOW_SIZE' : WINDOW_SIZE,
                                    'precision_bag'   : precision_bag})

        rf_clf = RandomForestClassifier(max_depth=best_parameters['rf_depth'], random_state=42)
        rf_clf.fit(X_train, y_train)
        cm_rf = confusion_matrix(rf_clf.predict(X_test), y_test)
        precision_rf   = calculate_general_precision(cm_rf)
        if precision_rf > best_parameters['precision_rf']:
            best_parameters.update({'rf_group_number': group_number,
                                    'rf_WINDOW_SIZE' : WINDOW_SIZE,
                                    'precision_rf'   : precision_rf})
            
        svm_clf = SVC(kernel="rbf", gamma= best_parameters['svm_gamma'], C=best_parameters['svm_C'])
        svm_clf.fit(X_train, y_train)
        cm_svm = confusion_matrix(svm_clf.predict(X_test), y_test)
        precision_svm  = calculate_general_precision(cm_svm)
        if precision_svm > best_parameters['precision_svm']:
            best_parameters.update({'svm_group_number': group_number,
                                    'svm_WINDOW_SIZE' : WINDOW_SIZE,
                                    'precision_svm'   : precision_svm})
        
        best_parameters.update({'precision_lda'   : 0})    
        lda_clf = LinearDiscriminantAnalysis()
        lda_clf.fit(X_train, y_train)
        cm_lda = confusion_matrix(lda_clf.predict(X_test), y_test)
        precision_lda  = calculate_general_precision(cm_lda)
        if precision_lda > best_parameters['precision_lda']:
            best_parameters.update({'lda_group_number': group_number,
                                    'lda_WINDOW_SIZE' : WINDOW_SIZE,
                                    'precision_lda'   : precision_lda})
            
        print('Progresso: ' + str(count*10) + '%')
        count = count + 1

Progresso: 10%
Progresso: 20%
Progresso: 30%
Progresso: 40%
Progresso: 50%
Progresso: 60%
Progresso: 70%
Progresso: 80%
Progresso: 90%
Progresso: 100%


In [65]:
best_parameters.update({'best_group_number': mode([best_parameters['tree_group_number'],
                                                   best_parameters['bag_group_number'],
                                                   best_parameters['rf_group_number'],
                                                   best_parameters['svm_group_number'],
                                                   best_parameters['lda_group_number']]),
                        'best_WINDOW_SIZE' : mode([best_parameters['tree_WINDOW_SIZE'],
                                                   best_parameters['bag_WINDOW_SIZE'],
                                                   best_parameters['rf_WINDOW_SIZE'],
                                                   best_parameters['svm_WINDOW_SIZE'],
                                                   best_parameters['lda_WINDOW_SIZE']])})

In [66]:
best_parameters

{'tree_group_number': 1,
 'tree_WINDOW_SIZE': 250,
 'precision_tree': 0.8277310924369747,
 'bag_group_number': 2,
 'bag_WINDOW_SIZE': 250,
 'precision_bag': 0.8109243697478992,
 'rf_group_number': 1,
 'rf_WINDOW_SIZE': 250,
 'precision_rf': 0.907563025210084,
 'svm_group_number': 1,
 'svm_WINDOW_SIZE': 250,
 'precision_svm': 0.9369747899159664,
 'lda_group_number': 2,
 'lda_WINDOW_SIZE': 250,
 'precision_lda': 0.7983193277310925,
 'tree_depth': 10,
 'bag_n_estimators': 110,
 'bag_max_samples': 150,
 'rf_depth': 12,
 'svm_gamma': 0.0001,
 'svm_C': 100,
 'best_group_number': 1,
 'best_WINDOW_SIZE': 250}

In [67]:
with open('best_parameters.txt', 'w') as file:
     file.write(json.dumps(best_parameters))